In [13]:
import pandas as pd
import numpy as np

from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
def dashboard_stats(shot_csv, point_csv, timestamps, player_name):
    total_serves = len(point_csv[point_csv['serverName'] == player_name])
    first_serve_in_count = len(point_csv[(point_csv['serverName'] == player_name) & (point_csv['firstServeIn'] == 1)])
    first_serve_won_count = len(point_csv[(point_csv['serverName'] == player_name) & (point_csv['firstServeIn'] == 1) & (point_csv['pointWonBy'] == player_name)])
    percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_serves > 0 else 0
    percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

    second_serve_total_count = len(point_csv[(point_csv['serverName'] == player_name) & (point_csv['firstServeIn'] == 0)])
    second_serve_in_count = len(point_csv[(point_csv['serverName'] == player_name) & (point_csv['firstServeIn'] == 0)& (point_csv['secondServeIn'] == 1)])
    second_serve_won_count = len(point_csv[(point_csv['serverName'] == player_name) & (point_csv['firstServeIn'] == 0)& (point_csv['secondServeIn'] == 1) & (point_csv['pointWonBy'] == player_name)])
    percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
    percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0

    count_is_ace = (point_csv[point_csv['serverName'] == player_name]['isAce']).sum()
    count_is_double_fault = ((point_csv['serverName'] == player_name) & (point_csv['serveResult'] == "Double Fault")).sum()
         

    first_serve_ad = np.sum(((shot_csv["firstServeIn"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Ad")).astype(float))
    first_serve_deuce = np.sum(((shot_csv["firstServeIn"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Deuce")).astype(float))
    second_serve_ad = np.sum(((shot_csv["secondServeIn"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Ad")).astype(float))
    second_serve_deuce = np.sum(((shot_csv["secondServeIn"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Deuce")).astype(float))
    double_fault_ad = np.sum(((shot_csv["isDoubleFault"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Ad")).astype(float))
    double_fault_deuce = np.sum(((shot_csv["isDoubleFault"] == 1) & (shot_csv["serverName"] == player_name) & (shot_csv["side"] == "Deuce")).astype(float))
    
    ad_total = (first_serve_ad.sum() + second_serve_ad.sum() + double_fault_ad.sum())
    deuce_total = (first_serve_deuce.sum() + second_serve_deuce.sum() + double_fault_deuce.sum())
    
    first_serve_ad_perc = np.round((first_serve_ad/ad_total)*100, 3)
    first_serve_deuce_perc = np.round((first_serve_deuce/deuce_total)*100, 3)
    second_serve_ad_perc = np.round((second_serve_ad/ad_total)*100, 3)
    second_serve_deuce_perc = np.round((second_serve_deuce/deuce_total)*100, 3)
    double_fault_ad_perc = np.round((double_fault_ad/ad_total)*100, 3)
    double_fault_deuce_perc = np.round((double_fault_deuce/deuce_total)*100, 3)
    
    # Added
    # Average rally count in each match
    average_rally_count = np.average(point_csv["rallyCount"]) 
    # three-shot rallies count
    three_shot_rallies = np.sum((point_csv["rallyCount"] == 3) & (point_csv["serverName"] == player_name)) 
    # total break points
    total_break_points = np.sum((point_csv['isBreakPoint'] == 1)) 
    # break points won
    break_points_won = np.sum((point_csv['isBreakPoint'] == 1) & (point_csv['pointWonBy'] == player_name )) 
    # break points percentage 
    break_points_percentage = break_points_won / total_break_points * 100 
    # total points on serve won
    total_points_serve_won = np.sum((point_csv['serverName'] == player_name) & (point_csv['pointWonBy'] == player_name)) 
    # game point on serve won / total game point on serve 
    total_break_points_serve = np.sum((point_csv['isBreakPoint'] == 1) & (point_csv["serverName"] == player_name))
    total_break_points_serve_won = np.sum((point_csv['isBreakPoint'] == 1) & (point_csv["serverName"] == player_name) & (point_csv['pointWonBy'] == player_name))
    percentage_game_point_serve_won = total_break_points_serve_won / total_break_points_serve * 100 
    # match duration
    duration = int(timestamps.iloc[-1]['pointEndTime'] - timestamps.head(1)['pointStartTime'])
    duration_format = timedelta(milliseconds=duration) 
    # total groundstrokes
    total_groundstrokes = np.sum((shot_csv['shotFhBh'] != "") & (shot_csv['isVolley'].isna()) & (shot_csv['isOverhead'].isna()) & (shot_csv['isApproach'].isna()) & (shot_csv['isDropshot'].isna()) & (shot_csv['isLob'].isna()) & (shot_csv['shotHitBy'] == player_name)) 
    # count of ground strokes that resulted in a win
    total_groundstrokes_winner =  np.sum((shot_csv['shotFhBh'] != "") & (shot_csv['isVolley'].isna()) & (shot_csv['isOverhead'].isna()) & (shot_csv['isApproach'].isna()) & (shot_csv['isDropshot'].isna()) & (shot_csv['isLob'].isna()) & (shot_csv['pointWonBy'] == player_name)) 
    # total serves
    total_serves = np.sum((point_csv['serverName'] == player_name)) 
    # total aces
    total_aces = np.sum((shot_csv['isAce'] == 1) & (shot_csv['serverName'] == player_name)) 
    print(point_csv['firstServeIn'])
    # total number of returns hit
    return_points = point_csv[(point_csv['returnerName'] == player_name)]
    total_return = len(return_points)
    # returns won count
    returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
    # volley count
    total_volleys = np.sum((shot_csv['isVolley'] == 1) & (shot_csv['shotHitBy'] == player_name)) 
    # volley winner count
    total_volleys_won = np.sum((shot_csv['isVolley'] == 1) & (shot_csv['pointWonBy'] == player_name)) 
    # at net points count
    total_at_net = 0 
    if str(shot_csv['player1Name'][1]) == player_name:
        total_at_net = int(np.sum(shot_csv['atNetPlayer1']))
    elif str(shot_csv['player2Name'][1]) == player_name:
        total_at_net = int(np.sum(shot_csv['atNetPlayer2']))
    # number of slices hit
    total_slices = np.sum((shot_csv['isSlice'] == 1) & (shot_csv['shotHitBy'] == player_name)) 
    # total number of dropshots won
    total_dropshots_won = np.sum((shot_csv['isDropshot'] == 1) & (shot_csv['pointWonBy'] == player_name))
    # total forehand and bachand errors (count)
    forehand_errors = point_csv[(point_csv['lastShotHitBy'] == player_name) &
                           (point_csv['lastShotResult'] == 'Error') &
                           (point_csv['lastShotFhBh'] == 'Forehand') &
                           (~point_csv['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN
 
    backhand_errors = point_csv[(point_csv['lastShotHitBy'] == player_name) &
                           (point_csv['lastShotResult'] == 'Error') &
                           (point_csv['lastShotFhBh'] == 'Backhand') &
                           (~point_csv['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN
    forehand_counts = forehand_errors.shape[0]  # Count rows
    backhand_counts = backhand_errors.shape[0]  # Count rows
    total_error_counts = forehand_counts + backhand_counts
    # total forehand and backhand errors (%)
    
    match_title = str(shot_csv["clientTeam"][0]) + "-" + str(shot_csv["player1Name"][1]) + " " + "vs." + " " + str(shot_csv["opponentTeam"][0]) + "-" + str(shot_csv["player2Name"][1])
    match_date = shot_csv["Date"][0]
    
    
    dashboard_data = {
        "Date" : match_date,
        "Match" : match_title,
        "Duration": str(duration_format).split(".")[0],
        "Player Name": player_name,
        "Total serves": total_serves,
        "Aces": count_is_ace,
        "1st Serve In %": round(percentage_first_serve_in, 2),
        "2nd Serve In %": round(percentage_second_serve_in, 2),
        "1st Serve Won %": round(percentage_first_serve_won, 2),
        "2nd Serve Won %": round(percentage_second_serve_won, 2),
        "Double Faults": count_is_double_fault,
        "1st serve Ad": first_serve_ad,
        "1st serve De": first_serve_deuce,
        "1st serve Ad %": round(first_serve_ad_perc, 2),
        "1st serve De %": round(first_serve_deuce_perc, 2),
        "2nd serve Ad %": round(second_serve_ad_perc, 2),
        "2nd serve De %": round(second_serve_deuce_perc, 2),
        "Double Fault Ad %": round(double_fault_ad_perc, 2),
        "Double Fault De %": round(double_fault_deuce_perc, 2),
        
        # Added
        "Average Rally Count": round(average_rally_count, 2),
        "3 Shot Rally Count": three_shot_rallies,
        "Break Points": total_break_points,
        "Break Points Won": break_points_won,
        "Break Points Won %": round(break_points_percentage, 2),
        "Total Points Won on Serve": total_points_serve_won,
        "Break Points Saved %": round(percentage_game_point_serve_won, 2),
        "Total Groundstrokes": total_groundstrokes,
        "Groundstrokes Won": total_groundstrokes_winner,
        "Total Returns": total_return,
        "Total Returns Won": returnWinner,
        "Volley Count": total_volleys,
        "Volley Winner Count": total_volleys_won,
        "At Net Count": total_at_net,
        "Total Slices": total_slices,
        "Number of Dropshots": total_dropshots_won,
        "Forehand/Backhand Errors (Count)": total_error_counts,
#         "Forehand Error %": round(forehand_counts / total_error_counts * 100, 2),
#         "Backhand Error %": round(backhand_counts / total_error_counts * 100, 2)
    }
    dashboard = pd.DataFrame(dashboard_data, index = [0])
    dashboard.set_index("Date", inplace = True)
    return dashboard

# Winter Quarter

## AC v Duke

In [26]:
shot_csv = pd.read_csv("Winter Q 24/AC v. Duke/Shot_Visuals_AnneLutkemeyer_KatieCodd.csv")
point_csv = pd.read_csv("Winter Q 24/AC v. Duke/Point_Visuals_AnneLutkemeyer_KatieCodd.csv")
timestamps = pd.read_csv("Winter Q 24/AC v. Duke/AC Duke Timestamps - Sheet1.csv")

ucla_dashboard_acduke = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_acduke = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_acduke = pd.concat([ucla_dashboard_acduke, opp_dashboard_acduke], axis = 1)

0      1
1      0
2      1
3      0
4      0
5      0
6      1
7      1
8      1
9      0
10     0
11     1
12     1
13     1
14     0
15     1
16     0
17     1
18     1
19     1
20     1
21     1
22     1
23     0
24     1
25     0
26     1
27     0
28     0
29     1
30     1
31     0
32     0
33     1
34     1
35     1
36     1
37     1
38     1
39     1
40     1
41     0
42     1
43     1
44     1
45     0
46     0
47     1
48     1
49     1
50     1
51     0
52     0
53     1
54     1
55     0
56     1
57     0
58     0
59     1
60     1
61     1
62     1
63     1
64     1
65     0
66     1
67     0
68     1
69     1
70     0
71     1
72     1
73     1
74     1
75     0
76     1
77     1
78     0
79     1
80     0
81     0
82     0
83     1
84     0
85     1
86     0
87     1
88     0
89     0
90     1
91     1
92     1
93     1
94     0
95     0
96     1
97     0
98     0
99     0
100    1
101    1
102    1
103    1
104    0
105    1
106    0
107    0
108    1
109    0
110    1
1

## Elise v. Duke

In [27]:
# shot_csv = pd.read_csv("Winter Q 24/Elise v. Duke/Shot_Visuals_EliseWagle_IuliiaBryzgalova.csv")
# point_csv = pd.read_csv("Winter Q 24/Elise v. Duke/Point_Visuals_EliseWagle_IuliiaBryzgalova.csv")
# timestamps = pd.read_csv("Winter Q 24/Elise v. Duke/TIMESTAMPS Elise v. Duke - Sheet1.csv")

# ucla_dashboard_eliseduke = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
# opp_dashboard_eliseduke = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

# dashboard_eliseduke = pd.concat([ucla_dashboard_eliseduke, opp_dashboard_eliseduke], axis = 1)

## Sasha v. LMU

In [31]:
shot_csv = pd.read_csv("Winter Q 24/Sasha v. LMU/Shot_Visuals_SashaVagramov_AnastasiaBozova.csv")
point_csv = pd.read_csv("Winter Q 24/Sasha v. LMU/Point_Visuals_SashaVagramov_AnastasiaBozova.csv")
timestamps = pd.read_csv("Winter Q 24/Sasha v. LMU/Sasha v. LMU Timestamps - Sheet1.csv")

ucla_dashboard_sashalmu = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_sashalmu = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_sashalmu = pd.concat([ucla_dashboard_sashalmu, opp_dashboard_sashalmu], axis = 1)

0     0
1     1
2     0
3     0
4     1
5     0
6     1
7     0
8     1
9     1
10    0
11    0
12    0
13    1
14    0
15    0
16    1
17    0
18    1
19    1
20    0
21    1
22    0
23    1
24    1
25    1
26    1
27    0
28    1
29    1
30    0
31    1
32    0
33    1
34    0
35    0
36    1
37    0
38    0
39    1
40    1
41    0
42    1
43    0
44    1
45    1
46    1
47    1
48    0
49    1
50    1
51    0
52    1
53    0
54    1
55    1
56    0
57    1
58    0
59    0
60    0
61    1
62    0
63    0
64    1
65    1
66    1
67    1
68    1
69    1
70    1
71    1
72    1
73    1
74    0
75    1
76    1
77    0
78    1
79    1
80    1
81    1
82    1
83    1
84    0
85    1
86    1
87    0
88    0
89    1
90    1
91    1
92    1
93    1
94    1
95    1
96    1
97    1
98    1
Name: firstServeIn, dtype: int64
0     0
1     1
2     0
3     0
4     1
5     0
6     1
7     0
8     1
9     1
10    0
11    0
12    0
13    1
14    0
15    0
16    1
17    0
18    1
19    1
20    0
21    1

# Spring Quarter

## Ahmani v Texas

In [32]:
shot_csv = pd.read_csv("Ahmani v. Texas/Shot_Visuals_AhmaniGuichard_VivianOvrootsky.csv")
point_csv = pd.read_csv("Ahmani v. Texas/Point_Visuals_AhmaniGuichard_VivianOvrootsky.csv")
timestamps = pd.read_csv("Ahmani v. Texas/Ahmani Texas Timestamps - Sheet1.csv")

ucla_dashboard_ahmanitexas = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_ahmanitexas = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_ahmanitexas = pd.concat([ucla_dashboard, opp_dashboard], axis = 1)


0      1
1      1
2      0
3      1
4      1
5      1
6      1
7      0
8      0
9      1
10     0
11     1
12     1
13     1
14     0
15     1
16     1
17     0
18     1
19     1
20     0
21     1
22     1
23     1
24     0
25     1
26     1
27     0
28     1
29     1
30     1
31     1
32     0
33     1
34     1
35     1
36     1
37     1
38     1
39     1
40     1
41     1
42     0
43     1
44     1
45     0
46     1
47     1
48     0
49     1
50     0
51     1
52     0
53     0
54     0
55     1
56     1
57     0
58     1
59     0
60     1
61     0
62     1
63     1
64     1
65     1
66     1
67     1
68     0
69     1
70     1
71     1
72     1
73     0
74     1
75     0
76     1
77     1
78     1
79     1
80     0
81     1
82     1
83     1
84     1
85     1
86     1
87     0
88     1
89     1
90     1
91     1
92     1
93     0
94     1
95     1
96     0
97     1
98     1
99     0
100    1
101    1
102    0
103    0
104    1
105    1
106    0
107    1
108    1
109    0
110    1
1

## AC v SDSU

In [33]:
shot_csv = pd.read_csv("AC v. SDSU/Shot_Visuals_AnneLutkemeyer_AndreeaVelcea.csv")
point_csv = pd.read_csv("AC v. SDSU/Point_Visuals_AnneLutkemeyer_AndreeaVelcea.csv")
timestamps = pd.read_csv("AC v. SDSU/AC vs. SDSU Timestamps - Sheet1.csv")

ucla_dashboard_acsdsu = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_acsdsu = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_acsdsu = pd.concat([ucla_dashboard_acsdsu, opp_dashboard_acsdsu], axis = 1)


0     0
1     1
2     0
3     1
4     1
5     1
6     0
7     0
8     1
9     1
10    1
11    1
12    1
13    0
14    0
15    1
16    1
17    0
18    1
19    0
20    0
21    1
22    1
23    0
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    0
32    0
33    1
34    1
35    0
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    0
47    0
48    1
49    1
50    0
51    1
52    1
53    0
54    1
55    1
56    1
57    1
58    1
59    1
60    1
61    1
62    1
63    0
64    1
65    0
66    0
67    0
68    0
69    0
70    0
71    1
Name: firstServeIn, dtype: int64
0     0
1     1
2     0
3     1
4     1
5     1
6     0
7     0
8     1
9     1
10    1
11    1
12    1
13    0
14    0
15    1
16    1
17    0
18    1
19    0
20    0
21    1
22    1
23    0
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    0
32    0
33    1
34    1
35    0
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    0
47    0
48    1

## Alex v Oregon

In [34]:
shot_csv = pd.read_csv("Alex v. Oregon/Shot_Visuals_AlexanderHoogmartens_LennLuemkemann.csv")
point_csv = pd.read_csv("Alex v. Oregon/Point_Visuals_AlexanderHoogmartens_LennLuemkemann.csv")
timestamps = pd.read_csv("Alex v. Oregon/Timestamps Alex v. Oregon - Sheet1.csv")

ucla_dashboard_alexoregon = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_alexoregon = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_alexoregon = pd.concat([ucla_dashboard_alexoregon, opp_dashboard_alexoregon], axis = 1)


0     1
1     0
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    0
17    1
18    1
19    0
20    0
21    0
22    1
23    0
24    1
25    1
26    1
27    1
28    0
29    1
30    0
31    1
32    0
33    0
34    1
35    0
36    1
37    1
38    1
39    0
40    0
41    0
42    1
43    1
44    1
45    1
46    0
47    0
48    1
49    1
50    1
51    0
52    1
53    1
54    1
55    0
56    1
57    1
58    1
59    0
60    0
61    1
62    1
63    0
64    1
65    1
66    1
67    1
68    1
69    1
70    0
71    1
72    1
73    1
74    1
75    0
76    0
77    0
78    0
79    1
80    1
81    0
82    1
83    1
84    0
85    1
86    0
87    1
88    1
89    1
90    1
91    1
92    0
93    0
94    1
95    1
Name: firstServeIn, dtype: int64
0     1
1     0
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    0
17    1
18    1
19    0
20    0
21    0
22    1
23    0
24    1

## Alex v USC

In [35]:
shot_csv = pd.read_csv("Alex v. USC/Shot_Visuals_AlexanderHoogmartens_KarlLee.csv")
point_csv = pd.read_csv("Alex v. USC/Point_Visuals_AlexanderHoogmartens_KarlLee.csv")
timestamps = pd.read_csv("Alex v. USC/Alexander Hoogmartens vs. Karl Lee USC Timestamps - Sheet1.csv")

ucla_dashboard_alexusc = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_alexusc = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_alexusc = pd.concat([ucla_dashboard_alexusc, opp_dashboard_alexusc], axis = 1)


0      1
1      1
2      1
3      0
4      0
5      1
6      1
7      1
8      0
9      1
10     0
11     1
12     0
13     0
14     1
15     0
16     1
17     0
18     0
19     1
20     1
21     1
22     1
23     0
24     1
25     1
26     0
27     1
28     0
29     1
30     1
31     0
32     0
33     1
34     0
35     0
36     1
37     1
38     0
39     1
40     1
41     0
42     1
43     0
44     1
45     0
46     1
47     1
48     0
49     1
50     1
51     0
52     1
53     1
54     0
55     0
56     1
57     1
58     0
59     1
60     1
61     1
62     1
63     1
64     0
65     0
66     1
67     1
68     1
69     0
70     1
71     1
72     1
73     0
74     0
75     1
76     0
77     1
78     0
79     0
80     1
81     0
82     1
83     1
84     1
85     0
86     1
87     1
88     1
89     0
90     0
91     1
92     1
93     1
94     1
95     0
96     1
97     0
98     0
99     1
100    0
101    1
102    1
103    0
104    1
105    1
106    0
107    1
108    1
109    0
110    1
1

## Giac v Oregon

In [36]:
shot_csv = pd.read_csv("Giac v. Oregon/Shot_Visuals_GiacomoRevelli_MatthewBurton.csv")
point_csv = pd.read_csv("Giac v. Oregon/Point_Visuals_GiacomoRevelli_MatthewBurton.csv")
timestamps = pd.read_csv("Giac v. Oregon/Giac vs. Oregon Timestamps - Sheet1.csv")

ucla_dashboard_giacoregon = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_giacoregon = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_giacoregon = pd.concat([ucla_dashboard_giacoregon, opp_dashboard_giacoregon], axis = 1)


0      0
1      0
2      0
3      0
4      1
5      0
6      1
7      0
8      1
9      1
10     0
11     0
12     1
13     1
14     0
15     0
16     1
17     0
18     0
19     0
20     1
21     1
22     1
23     0
24     1
25     1
26     0
27     0
28     0
29     1
30     1
31     1
32     1
33     1
34     0
35     1
36     0
37     0
38     0
39     1
40     0
41     1
42     1
43     0
44     1
45     0
46     0
47     1
48     1
49     1
50     1
51     1
52     0
53     1
54     1
55     1
56     1
57     1
58     1
59     1
60     1
61     1
62     1
63     0
64     0
65     1
66     0
67     1
68     0
69     0
70     1
71     1
72     1
73     0
74     0
75     0
76     1
77     0
78     0
79     0
80     1
81     1
82     0
83     1
84     1
85     1
86     1
87     1
88     1
89     1
90     0
91     1
92     1
93     1
94     1
95     1
96     1
97     1
98     1
99     1
100    1
101    1
102    1
103    0
104    1
105    0
106    1
107    1
108    0
109    1
110    1
1

## Giac v UNLV

In [37]:
shot_csv = pd.read_csv("Giac v. UNLV/Shot_Visuals_GiacomoRevelli_IlliaMaksymchuk.csv")
point_csv = pd.read_csv("Giac v. UNLV/Point_Visuals_GiacomoRevelli_IlliaMaksymchuk.csv")
timestamps = pd.read_csv("Giac v. UNLV/Giacomo Revelli vs. UNLV Timestamps - Sheet1.csv")

ucla_dashboard_giacunlv = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_giacunlv = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_giacunlv = pd.concat([ucla_dashboard_giacunlv, opp_dashboard_giacunlv], axis = 1)


0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      1
8      0
9      1
10     1
11     1
12     1
13     1
14     1
15     0
16     0
17     1
18     1
19     1
20     1
21     0
22     0
23     1
24     0
25     1
26     1
27     1
28     1
29     1
30     0
31     1
32     1
33     1
34     0
35     0
36     1
37     1
38     1
39     1
40     0
41     0
42     1
43     1
44     1
45     0
46     1
47     1
48     0
49     0
50     0
51     0
52     1
53     1
54     1
55     0
56     1
57     1
58     1
59     0
60     0
61     1
62     1
63     0
64     1
65     1
66     1
67     1
68     1
69     1
70     1
71     0
72     0
73     1
74     1
75     1
76     0
77     0
78     0
79     0
80     0
81     1
82     1
83     1
84     0
85     1
86     1
87     1
88     1
89     1
90     1
91     0
92     0
93     1
94     1
95     1
96     0
97     0
98     1
99     1
100    1
Name: firstServeIn, dtype: int64
0      1
1      0
2      1
3      1
4      1
5      1
6   

## Spencer v Stanford

In [38]:
shot_csv = pd.read_csv("Spencer v. Stanford/Shot_Visuals_SpencerJohnson_SamirBanerjee.csv")
point_csv = pd.read_csv("Spencer v. Stanford/Point_Visuals_SpencerJohnson_SamirBanerjee.csv")
timestamps = pd.read_csv("Spencer v. Stanford/Spencer Johnson vs. Samir Banerjee Stanford Timestamps.csv")

ucla_dashboard_spencerstanford = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_spencerstanford = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_spencerstanford = pd.concat([ucla_dashboard_spencerstanford, opp_dashboard_spencerstanford], axis = 1)


0      1
1      1
2      0
3      1
4      0
5      1
6      1
7      1
8      1
9      0
10     1
11     0
12     1
13     1
14     0
15     1
16     1
17     1
18     0
19     0
20     1
21     1
22     1
23     0
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     0
32     1
33     1
34     0
35     0
36     1
37     1
38     1
39     1
40     1
41     1
42     1
43     0
44     1
45     1
46     1
47     1
48     1
49     1
50     1
51     0
52     1
53     0
54     1
55     1
56     1
57     1
58     0
59     1
60     1
61     1
62     1
63     0
64     0
65     0
66     1
67     1
68     0
69     1
70     1
71     1
72     1
73     1
74     1
75     0
76     0
77     0
78     1
79     1
80     1
81     1
82     1
83     1
84     0
85     1
86     1
87     1
88     1
89     1
90     1
91     0
92     1
93     1
94     1
95     0
96     1
97     1
98     0
99     1
100    0
101    1
102    1
103    1
104    1
105    0
106    1
107    1
108    1
109    0
110    0
1

## Spencer v UCI

In [39]:
shot_csv = pd.read_csv("Spencer v. UCI/Shot_Visuals_SpencerJohnson_NoahZamora.csv")
point_csv = pd.read_csv("Spencer v. UCI/Point_Visuals_SpencerJohnson_NoahZamora.csv")
timestamps = pd.read_csv("Spencer v. UCI/Spencer vs. UCI timestamps.csv")

ucla_dashboard_spenceruci = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player1Name'][1])
opp_dashboard_spenceruci = dashboard_stats(shot_csv, point_csv, timestamps, point_csv['player2Name'][1])

dashboard_spenceruci = pd.concat([ucla_dashboard_spenceruci, opp_dashboard_spenceruci], axis = 1)


0      1
1      1
2      1
3      0
4      0
5      1
6      1
7      1
8      1
9      0
10     1
11     1
12     0
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     0
23     1
24     0
25     1
26     1
27     1
28     0
29     0
30     0
31     0
32     0
33     1
34     1
35     1
36     1
37     1
38     0
39     1
40     1
41     1
42     0
43     0
44     0
45     1
46     0
47     1
48     1
49     0
50     1
51     0
52     1
53     1
54     0
55     1
56     0
57     0
58     1
59     1
60     0
61     0
62     1
63     0
64     0
65     1
66     1
67     1
68     1
69     1
70     0
71     1
72     1
73     1
74     1
75     1
76     1
77     1
78     0
79     1
80     1
81     1
82     1
83     1
84     1
85     1
86     1
87     0
88     0
89     0
90     1
91     1
92     1
93     1
94     0
95     0
96     1
97     1
98     0
99     1
100    1
101    1
Name: firstServeIn, dtype: int64
0      1
1      1
2      1
3      0
4      0
5   

# Full Dashboard

In [44]:
main_dashboard = pd.concat([dashboard_acduke, dashboard_sashalmu, dashboard_ahmanitexas, dashboard_acsdsu, dashboard_alexoregon, dashboard_giacoregon, dashboard_giacunlv, dashboard_spencerstanford, dashboard_spenceruci])

main_dashboard.sort_index(axis = 0, ascending=True, inplace=True)
main_dashboard

,Match,Duration,Player Name,Total serves,Aces,1st Serve In %,2nd Serve In %,1st Serve Won %,2nd Serve Won %,Double Faults,1st serve Ad,1st serve De,1st serve Ad %,1st serve De %,2nd serve Ad %,2nd serve De %,Double Fault Ad %,Double Fault De %,Average Rally Count,3 Shot Rally Count,Break Points,Break Points Won,Break Points Won %,Total Points Won on Serve,Break Points Saved %,Total Groundstrokes,Groundstrokes Won,Total Returns,Total Returns Won,Volley Count,Volley Winner Count,At Net Count,Total Slices,Number of Dropshots,Forehand/Backhand Errors (Count),Match,Duration,Player Name,Total serves,Aces,1st Serve In %,2nd Serve In %,1st Serve Won %,2nd Serve Won %,Double Faults,1st serve Ad,1st serve De,1st serve Ad %,1st serve De %,2nd serve Ad %,2nd serve De %,Double Fault Ad %,Double Fault De %,Average Rally Count,3 Shot Rally Count,Break Points,Break Points Won,Break Points Won %,Total Points Won on Serve,Break Points Saved %,Total Groundstrokes,Groundstrokes Won,Total Returns,Total Returns Won,Volley Count,Volley Winner Count,At Net Count,Total Slices,Number of Dropshots,Forehand/Backhand Errors (Count)
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2/23/24,UCLA-Sasha Vagramov vs. LMU-Anastasia Bozova,1:10:45,Sasha Vagramov,53,1,62.26,55.00,72.73,72.73,9,17.0,16.0,65.39,61.54,15.38,26.92,19.23,11.54,4.07,3,11,4,36.36,32,50.00,185,49,46,1,8,1,17,1,1,24,UCLA-Sasha Vagramov vs. LMU-Anastasia Bozova,1:10:45,Anastasia Bozova,46,2,65.22,75.00,60.00,41.67,4,10.0,20.0,45.46,83.33,40.91,12.50,13.64,4.17,4.07,8,11,7,63.64,23,66.67,193,40,53,4,0,3,2,9,1,40
2/25/24,UCLA-Giacomo Revelli vs. UNLV-Illia Maksymchuk,1:21:27,Giacomo Revelli,48,4,75.00,100.00,72.22,75.00,0,17.0,19.0,70.83,79.17,29.17,20.83,0.00,0.00,5.75,2,5,3,60.00,35,100.00,260,50,53,2,6,4,20,15,0,29,UCLA-Giacomo Revelli vs. UNLV-Illia Maksymchuk,1:21:27,Illia Maksymchuk,53,1,60.38,95.24,59.38,45.00,2,15.0,17.0,53.57,68.00,46.43,28.00,0.00,4.00,5.75,6,5,2,40.00,28,50.00,250,31,48,0,14,5,42,21,1,38
2/3/24,UCLA-Anne Lutkemeyer vs. Duke-Katie Codd,1:43:51,Anne Lutkemeyer,63,0,66.67,57.14,61.90,75.00,9,16.0,26.0,51.61,81.25,29.03,12.50,19.36,6.25,5.80,8,13,9,69.23,36,66.67,327,56,69,3,15,7,19,12,1,44,UCLA-Anne Lutkemeyer vs. Duke-Katie Codd,1:43:51,Katie Codd,69,0,60.87,77.78,52.38,42.86,6,24.0,18.0,75.00,50.00,21.88,38.89,3.12,11.11,5.80,6,13,4,30.77,32,25.00,344,51,63,0,4,3,12,15,0,38
3/7/24,UCLA-Spencer Johnson vs. UC Irvine-Noah Zamora,1:19:41,Spencer Johnson,50,4,72.00,78.57,66.67,54.55,3,20.0,16.0,86.96,59.26,8.70,33.33,4.35,7.41,4.90,15,12,4,33.33,30,60.00,218,38,52,0,16,5,15,6,2,35,UCLA-Spencer Johnson vs. UC Irvine-Noah Zamora,1:19:41,Noah Zamora,52,1,63.46,89.47,75.76,47.06,2,19.0,14.0,73.08,53.85,19.23,46.15,7.69,0.00,4.90,6,12,8,66.67,33,85.71,231,44,50,1,6,8,3,18,0,35
4/12/24,UCLA-Spencer Johnson vs. Stanford-Samir Banerjee,2:13:44,Spencer Johnson,93,13,76.34,90.91,66.20,65.00,2,32.0,39.0,74.42,78.00,20.93,22.00,4.65,0.00,4.08,19,37,19,51.35,60,55.56,346,77,99,1,19,6,26,16,3,64,UCLA-Spencer Johnson vs. Stanford-Samir Banerjee,2:13:44,Samir Banerjee,99,4,70.71,86.21,61.43,68.00,4,34.0,37.0,69.39,74.00,26.53,26.00,4.08,0.00,4.08,13,37,18,48.65,61,50.00,362,78,93,5,11,11,7,12,2,57
4/5/24,UCLA-Alexander Hoogmartens vs. Oregon-Lenn Lue...,1:07:55,Alexander Hoogmartens,49,5,67.35,81.25,66.67,84.62,3,16.0,17.0,64.00,70.83,28.00,25.00,8.00,4.17,4.44,6,20,9,45.00,33,83.33,202,51,47,1,4,2,0,7,0,26,UCLA-Alexander Hoogmartens vs. Oregon-Lenn Lue...,1:07:55,Lenn Luemkemann,47,1,65.96,68.75,61.29,27.27,5,10.0,21.0,47.62,84.00,42.86,8.00,9.52,8.00,4.44,12,20,11,55.00,23,71.43,200,34,49,0,3,2,3,8,0,34
4/5/24,UCLA-Giacomo Revelli vs. Oregon-Matthew Burton,2:09:16,Giacomo Revelli,92,1,60.87,88.89,66.07,43.75,4,26.0,30.0,57.78,63.83,35.56,34.04,6.67,2.13,7.25,10,22,16,72.73,51,71.43,584,70,79,2,9,7,21,37,2,60,UCLA-Giacomo Revelli vs. Oregon-Matthew Burton,2:09:16,Matthew Burton,79,1,62.03,83.33,63.27,40.00,5,25.0,24.0,65.79,60.00,28.95,35.00,5.26,5.00,7.25,5,

In [48]:
import pandas as pd

# Combine the dashboards into one DataFrame
main_dashboard = pd.concat([
    dashboard_acduke, dashboard_sashalmu, dashboard_ahmanitexas,
    dashboard_acsdsu, dashboard_alexoregon, dashboard_giacoregon,
    dashboard_giacunlv, dashboard_spencerstanford, dashboard_spenceruci
])

# Ensure the 'date' column is in datetime format
main_dashboard['Date'] = pd.to_datetime(main_dashboard['Date'])

# Sort the DataFrame by the 'date' column
main_dashboard.sort_values(by='Date', ascending=True, inplace=True)

# Display the sorted DataFrame
print(main_dashboard)

KeyError: 'Date'